<a href="https://colab.research.google.com/github/rubyclaguna/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/Copy_of_LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
# Upload data
import pandas as pd 
from google.colab import files
uploaded = files.upload()

Saving house-votes-84.data to house-votes-84 (1).data


In [0]:
# no specific column names assigned, soi set header=None
df = pd.read_csv('house-votes-84.data', header = None)
print(df.shape)
df.head()

(435, 17)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
# Assign column names 
df = df.rename ({0: 'party',           
                 1: 'handicapped-infants',  
                 2: 'water-project-cost-sharing',
                 3: 'adoption-of-the-budget-resolution', 
                 4: 'physician-fee-freeze', 
                 5: 'el-salvador-aid', 
                 6: 'religious-groups-in-schools', 
                 7: 'anti-satellite-test-ban', 
                 8: 'aid-to-nicaraguan-contras', 
                 9: 'mx-missile',
                10: 'immigration',
                11: 'synfuels-corporation-cutback',
                12: 'education-spending',
                13: 'superfund-right-to-sue',
                14: 'crime',
                15: 'duty-free-exports',
                16: 'export-administration-act-south-africa'}, axis = 1)

In [0]:
df.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 17 columns):
party                                     435 non-null object
handicapped-infants                       435 non-null object
water-project-cost-sharing                435 non-null object
adoption-of-the-budget-resolution         435 non-null object
physician-fee-freeze                      435 non-null object
el-salvador-aid                           435 non-null object
religious-groups-in-schools               435 non-null object
anti-satellite-test-ban                   435 non-null object
aid-to-nicaraguan-contras                 435 non-null object
mx-missile                                435 non-null object
immigration                               435 non-null object
synfuels-corporation-cutback              435 non-null object
education-spending                        435 non-null object
superfund-right-to-sue                    435 non-null object
crime                      

In [0]:
df.isnull().sum()

party                                     0
handicapped-infants                       0
water-project-cost-sharing                0
adoption-of-the-budget-resolution         0
physician-fee-freeze                      0
el-salvador-aid                           0
religious-groups-in-schools               0
anti-satellite-test-ban                   0
aid-to-nicaraguan-contras                 0
mx-missile                                0
immigration                               0
synfuels-corporation-cutback              0
education-spending                        0
superfund-right-to-sue                    0
crime                                     0
duty-free-exports                         0
export-administration-act-south-africa    0
dtype: int64

In [0]:
import numpy as np
# Change '?' to NaN to determine missing values. 
df = df.replace ('?', np.NaN)

In [0]:
df.isnull().sum()

party                                       0
handicapped-infants                        12
water-project-cost-sharing                 48
adoption-of-the-budget-resolution          11
physician-fee-freeze                       11
el-salvador-aid                            15
religious-groups-in-schools                11
anti-satellite-test-ban                    14
aid-to-nicaraguan-contras                  15
mx-missile                                 22
immigration                                 7
synfuels-corporation-cutback               21
education-spending                         31
superfund-right-to-sue                     25
crime                                      17
duty-free-exports                          28
export-administration-act-south-africa    104
dtype: int64

In [0]:
# Drop missing values. Should be enough to data to work with if these values are dropped. 
df_all = df.dropna()

In [0]:
df_all = df_all.replace({'n':0, 'y':1})
df_all.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
5,democrat,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
8,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
19,democrat,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1
23,democrat,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1
25,democrat,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1


In [0]:
# Seperate by party in order to compare data
df_dem = df[df.party == "democrat"]2
df_rep = df[df.party == "republican"]

In [0]:
group = df_all.groupby(["party"]).mean()
group = group.T
group

party,democrat,republican
handicapped-infants,0.588710,0.212963
water-project-cost-sharing,0.451613,0.472222
adoption-of-the-budget-resolution,0.854839,0.157407
physician-fee-freeze,0.048387,0.990741
el-salvador-aid,0.201613,0.953704
religious-groups-in-schools,0.443548,0.870370
anti-satellite-test-ban,0.766129,0.268519
aid-to-nicaraguan-contras,0.830645,0.148148
mx-missile,0.790323,0.138889
immigration,0.532258,0.574074


In [0]:
# Add 3rd column to compare difference 
group['diff'] = group['democrat'] - group['republican']

In [0]:
group

party,democrat,republican,diff
handicapped-infants,0.588710,0.212963,0.375747
water-project-cost-sharing,0.451613,0.472222,-0.020609
adoption-of-the-budget-resolution,0.854839,0.157407,0.697431
physician-fee-freeze,0.048387,0.990741,-0.942354
el-salvador-aid,0.201613,0.953704,-0.752091
religious-groups-in-schools,0.443548,0.870370,-0.426822
anti-satellite-test-ban,0.766129,0.268519,0.497611
aid-to-nicaraguan-contras,0.830645,0.148148,0.682497
mx-missile,0.790323,0.138889,0.651434
immigration,0.532258,0.574074,-0.041816


In [0]:
import scipy
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
# Lowest p-value for republican support over democrat
scipy.stats.ttest_ind(df_rep['physician-fee-freeze'], df_dem['physician-fee-freeze'])

TypeError: ignored